### Recurrent Neural Network (Many to One)

In [18]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd

In [5]:
# Hyper Parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

In [6]:
# MNIST Dataset
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

Processing...
Done!


In [7]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda()) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

In [8]:
rnn = RNN(input_size, hidden_size, num_layers, num_classes)
rnn.cuda()

RNN (
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (fc): Linear (128 -> 10)
)

In [9]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [10]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [11]:
my_labels = []
my_predictions = []

# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size)).cuda()  
        labels = Variable(labels).cuda()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/2], Step [100/600], Loss: 0.4742
Epoch [1/2], Step [200/600], Loss: 0.2538
Epoch [1/2], Step [300/600], Loss: 0.2249
Epoch [1/2], Step [400/600], Loss: 0.1433
Epoch [1/2], Step [500/600], Loss: 0.0512
Epoch [1/2], Step [600/600], Loss: 0.0983
Epoch [2/2], Step [100/600], Loss: 0.1247
Epoch [2/2], Step [200/600], Loss: 0.1738
Epoch [2/2], Step [300/600], Loss: 0.0948
Epoch [2/2], Step [400/600], Loss: 0.0966
Epoch [2/2], Step [500/600], Loss: 0.0482
Epoch [2/2], Step [600/600], Loss: 0.0485


In [16]:
my_labels = []
my_predictions = []

# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size)).cuda()
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()
    my_labels += labels.tolist()
    my_predictions += predicted.cpu().tolist()

print('Test Accuracy of the model on the 10000 test images: %d %%\n' % (100 * correct / total)) 

print('%d out of %d classified correctly' % (correct, total))

Test Accuracy of the model on the 10000 test images: 97 %

9706 out of 10000 classified correctly


In [20]:
df = pd.DataFrame({})
df['label'] = my_labels
df['prediction'] = my_predictions
df[df['label'] != df['prediction']].head(10)

,label,prediction
8,5,6
72,2,3
115,4,6
149,2,3
184,8,3
247,4,2
300,4,6
321,2,7
337,7,3
358,7,4


In [ ]:
# Save the Model
torch.save(rnn.state_dict(), 'rnn.pkl')